# Genome Processing & Annotation

- This notebook generates annotations for the genes present in the MSK IMPACT dataset, in reference to GrCh37 build.

IMPACT dataset: 

For more information regarding collection methods, etc., see: 
- https://www.mskcc.org/msk-impact
- https://datacatalog.mskcc.org/dataset/10438

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

/Users/samgould/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
filepath = '/Volumes/Sam_G_SSD/2020-06-16-MSK-IMPACT_EDITED.txt'
impact_data = pd.read_csv(filepath, sep='\t')

/Users/samgould/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (45,48,88) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
impact_data

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,VARIANT_CLASS,all_effects,amino_acid_change,cDNA_Change,cDNA_position,cdna_change,comments,n_depth,t_depth,transcript
0,BRCA2,675,MSKCC,GRCh37,13,32937315,32937315,+,splice_acceptor_variant,Splice_Site,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRCA2,0,MSKCC,37,13,32914437,32914438,+,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MUTYH,4595,MSKCC,GRCh37,1,45798475,45798475,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BRCA2,675,MSKCC,GRCh37,13,32893302,32893302,+,frameshift_variant,Frame_Shift_Ins,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRCA1,0,MSKCC,37,17,41251824,41251825,+,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422817,SMARCA4,6597,MSKCC,GRCh37,19,11144132,11144132,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422818,BRAF,673,MSKCC,GRCh37,7,140453149,140453149,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422819,NRAS,4893,MSKCC,GRCh37,1,115258747,115258747,+,missense_variant,Missense_Mutation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422820,TERT,7015,MSKCC,GRCh37,5,1295521,1295521,+,upstream_gene_variant,5'Flank,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
unique_genes = np.unique(np.asarray(impact_data['Hugo_Symbol']))

# Loading in Reference Genome GRCh37

- can check for PAM sequences on either side of the mutation (+/- strand?)
- Downloading reference sequence from: https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.26/ (assembly GRCh38)
- GRCh37: https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.25


In [8]:
from Bio import SeqIO
import gzip

In [9]:
#MSK IMPACT dataset uses GrCh37 build
np.unique(np.asarray(impact_data['NCBI_Build']))

array(['37', 'GRCh37'], dtype=object)

In [10]:
file = '/Volumes/Sam_G_SSD/GRCh37/ncbi-genomes-2022-03-17/GCF_000001405.25_GRCh37.p13_genomic.fna.gz'

with gzip.open(file, "rt") as handle:
    records = list(SeqIO.parse(handle, "fasta")) #about 4 Gb in  memory
    #records = list that contains sequences split up by chromosome (and intrachromosome splits up to some size)

In [11]:
#filtering out alternative sequences to only select consensus matches


wrong = ["alternate", "unplaced", "unlocalized", "patch"]
badlist = []
for key in wrong:
    for i in records:
        ii = i.description
        if key in ii:
            badlist.append(ii)
            
filtered = []
index_list = []
for idx, i in enumerate(records):
    ii = i.description
    if ii not in badlist:
        filtered.append(ii)
        index_list.append(idx)
        
filtered
    

['NC_000001.10 Homo sapiens chromosome 1, GRCh37.p13 Primary Assembly',
 'NC_000002.11 Homo sapiens chromosome 2, GRCh37.p13 Primary Assembly',
 'NC_000003.11 Homo sapiens chromosome 3, GRCh37.p13 Primary Assembly',
 'NC_000004.11 Homo sapiens chromosome 4, GRCh37.p13 Primary Assembly',
 'NC_000005.9 Homo sapiens chromosome 5, GRCh37.p13 Primary Assembly',
 'NC_000006.11 Homo sapiens chromosome 6, GRCh37.p13 Primary Assembly',
 'NC_000007.13 Homo sapiens chromosome 7, GRCh37.p13 Primary Assembly',
 'NC_000008.10 Homo sapiens chromosome 8, GRCh37.p13 Primary Assembly',
 'NC_000009.11 Homo sapiens chromosome 9, GRCh37.p13 Primary Assembly',
 'NC_000010.10 Homo sapiens chromosome 10, GRCh37.p13 Primary Assembly',
 'NC_000011.9 Homo sapiens chromosome 11, GRCh37.p13 Primary Assembly',
 'NC_000012.11 Homo sapiens chromosome 12, GRCh37.p13 Primary Assembly',
 'NC_000013.10 Homo sapiens chromosome 13, GRCh37.p13 Primary Assembly',
 'NC_000014.8 Homo sapiens chromosome 14, GRCh37.p13 Primary A

In [11]:
chromosome = 20
#22 (23-1) = X
#23 (24-1) = Y
#24 (25-1) = mitochondrial DNA
records[index_list[chromosome-1]]

SeqRecord(seq=Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN', SingleLetterAlphabet()), id='NC_000020.10', name='NC_000020.10', description='NC_000020.10 Homo sapiens chromosome 20, GRCh37.p13 Primary Assembly', dbxrefs=[])

In [13]:
k=902
print('ref = ' + impact_data.iloc[[k]]['Reference_Allele'].values[0])
#print('al1 = ' + impact_data.iloc[[k]]['Tumor_Seq_Allele1'].values[0])
#print('al2 = ' + impact_data.iloc[[k]]['Tumor_Seq_Allele2'].values[0])
seq_start = impact_data.iloc[[k]]['Start_Position'].values[0]
seq_end = impact_data.iloc[[k]]['End_Position'].values[0]
chromosome = impact_data.iloc[[k]]['Chromosome'].values[0]
seq1 = records[index_list[int(chromosome)-1]].seq
seq1[seq_start-1: seq_end]

ref = TCATCG


Seq('TCATCG', SingleLetterAlphabet())

Sequence start is offset by 1 (i.e. need to subtract 1 from start position to get true sequence information). 

# Accessing genome coordinates from gene name

https://medium.com/intothegenomics/annotate-genes-and-genomic-coordinates-using-python-9259efa6ffc2

https://blog.liang2.tw/posts/2018/06/gene-annotation-using-gffutils/


In [2]:
import gffutils

In [4]:
#MOVED THESE FILES AND the CREATED DIRECTORY FILE TO EXTERNAL SSD!!!
file = '/Users/samgould/Desktop/FSR Lab/2022-03-17/gencode.v19.annotation.gtf_withproteinids'

db = gffutils.create_db(
    file,
    dbfn='gencode_v19.db',
    verbose=True,
    merge_strategy='error',
    disable_infer_transcripts=True,
    disable_infer_genes=True,
)

2022-03-18 11:45:41,842 - INFO - Committing changes: 2619000 features
2022-03-18 11:45:43,637 - INFO - Populating features table and first-order relations: 2619443 features
2022-03-18 11:45:43,641 - INFO - Creating relations(parent) index
2022-03-18 11:45:57,811 - INFO - Creating relations(child) index
2022-03-18 11:46:13,210 - INFO - Creating features(featuretype) index
2022-03-18 11:46:54,272 - INFO - Creating features (seqid, start, end) index
2022-03-18 11:47:16,452 - INFO - Creating features (seqid, start, end, strand) index
2022-03-18 11:47:38,770 - INFO - Running ANALYZE features


In [87]:
tx1 = db['ENST00000366541.3']; 
tx1.chrom, tx1.start, tx1.end, tx1.strand
#tx1.attributes.items()

('chr1', 243419358, 243663394, '+')

In [106]:
tx = db['ENST00000269305.4'];
tx.attributes.items()[9][1][0]

'ENSP00000269305.4'

In [100]:
tx.end

7590856

In [101]:
gene = db['ENSG00000141510.11']
gene.end

7590856

Accessing the transcript coordinates for each gene by getting the start, end and strand of each gene.

I'm only getting the transcript coordinates, so not the 3' or 5' UTRs right now (need the gene id, which isn't provided here). I'll check if any of the mutations fall outside of these regions as well.

In [76]:
unique_genes = np.unique(np.asarray(impact_data['Hugo_Symbol']))

id_list = []

for i in unique_genes:
    k = np.asarray(impact_data[impact_data['Hugo_Symbol'] == i]['HGVSc'])
    #print(k)
    len(k)
    for idx, j in enumerate(k):
        if type(j)==str:
            id_list.append(j)
            #print(j)
            break
                #print(j)
        else:
            #print('false')
            if (idx+1)==len(k):
                print(str(i) + 'NO ID')
                #missing gene is'SDCCAG8'; id = ENST00000366541.3
                id_list.append('ENST00000366541.3')
            else:
                continue
            
                
    

SDCCAG8NO ID


In [77]:
len(id_list) #good; matches unique_genes

594

In [ ]:
#now splitting before : to remove SNP information
tx_ids = [i.split(':')[0] for i in id_list]

#and now getting lists of attributes
chrom = []
tx_start = []
tx_end = []
gene_start = []
gene_end = []
strand = []
gene_id = []
for i in tx_ids:
    tx = db[i]
    geneid = tx.attributes.items()[0][1][0]
    gene_id.append(geneid)
    
    
    #and now properties of gene
    gene = db[geneid]
    gene_chrom = gene.chrom
    genestart = gene.start
    geneend = gene.end
    
    
    chrom.append(gene_chrom)
    gene_start.append(genestart)
    gene_end.append(geneend)
    
    #and properties of transcript
    txstart = tx.start
    txend = tx.end
    txstrand = tx.strand
    
    tx_start.append(txstart)
    tx_end.append(txend)
    strand.append(txstrand)
    



In [116]:
#collating everything into a pandas dataframe and saving as csv for future use

gene_info = pd.DataFrame(list(zip(unique_genes, gene_id, tx_ids, 
                          chrom, gene_start, gene_end, tx_start, tx_end, strand)),
               columns =['gene', 'gene_id', 'transcript_id', 
                          'chrom', 'gene_start', 'gene_end', 'transcript_start', 
                         'transcript_end', 'strand'])

#save this 
#gene_info.to_csv(...)

# Loading in gene coordinate information

In [3]:
filename1 = '/Users/samgould/Desktop/FSR Lab/2022-03-17/gene_info.csv'
df1 = pd.read_csv(filename1)
df1

,gene,gene_id,transcript_id,chrom,gene_start,gene_end,transcript_start,transcript_end,strand
0,ABL1,ENSG00000097007.13,ENST00000318560.5,chr9,133589333,133763062,133710453,133763062,+
1,AC004906.3,ENSG00000237286.1,ENST00000423194.1,chr7,2983669,2986725,2983669,2986725,+
2,AC008738.1,ENSG00000230259.2,ENST00000425420.2,chr19,33790853,33793430,33790853,33793430,-
3,ACTG1,ENSG00000184009.5,ENST00000575842.1,chr17,79476997,79490873,79477015,79479807,-
4,ACVR1,ENSG00000115170.9,ENST00000263640.3,chr2,158592958,158732374,158592958,158731623,-
...,...,...,...,...,...,...,...,...,...
589,XRCC2,ENSG00000196584.2,ENST00000359321.1,chr7,152341864,152373250,152343589,152373250,-
590,YAP1,ENSG00000137693.9,ENST00000282441.5,chr11,101981192,102104154,101981192,102104154,+
591,YES1,ENSG00000176105.9,ENST00000314574.4,chr18,721588,812547,721748,812239,-
592,ZFHX3,ENSG00000140836.10,ENST00000268489.5,chr16,72816784,73093597,72816784,73082274,-
